In [52]:
import tensorflow as tf; print(f"\t\t– TENSORFLOW VERSION: {tf.__version__}");
import pandas as pd; pd.options.mode.chained_assignment = None;
import os
import ast
import cv2
import numpy as np
from glob import glob
from PIL import Image
import plotly.express as px
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm; tqdm.pandas();

		– TENSORFLOW VERSION: 2.9.0


In [53]:
DATA_DIR = "D:Dataset/dataset/hubmap-organ-segmentation/"
for file in tf.io.gfile.glob(os.path.join(DATA_DIR, "*")): print(f"\t--> {file}")

	--> D:Dataset\dataset\hubmap-organ-segmentation\masked_images
	--> D:Dataset\dataset\hubmap-organ-segmentation\sample_submission.csv
	--> D:Dataset\dataset\hubmap-organ-segmentation\test.csv
	--> D:Dataset\dataset\hubmap-organ-segmentation\test_images
	--> D:Dataset\dataset\hubmap-organ-segmentation\train.csv
	--> D:Dataset\dataset\hubmap-organ-segmentation\train_annotations
	--> D:Dataset\dataset\hubmap-organ-segmentation\train_images


In [77]:
train_df = pd.read_csv(DATA_DIR+"train.csv")
test_df = pd.read_csv(DATA_DIR+"test.csv")

In [44]:
def rle2mask(rle, width, target_size=None):
    if target_size == None:
        target_size = width

    rle = np.array(list(map(int, rle.split())))
    label = np.zeros((width*width))
    
    for start, end in zip(rle[::2], rle[1::2]):
        label[start:start+end] = 1
        
    #Convert label to image
    label = Image.fromarray(label.reshape(width, width))
    #Resize label
    label = label.resize((target_size, target_size))
    label = np.array(label).astype(float)
    #rescale label
    label = np.round((label - label.min())/(label.max() - label.min()))
    
    return label.T

In [82]:
tf.config.optimizer.set_jit(True) # XLA은 텐서플로우 환경안에서 도메인 별로 특화된 선형대수 계산 컴파일러이며, 연산 속도와 메모리 사용량을 향상시킨다.

In [85]:
for idx, row in train_df[:1].iterrows():
    size = row["img_height"]
    y = np.zeros((size, size,3))
    temp = row["rle"]
    rle = rle2mask(temp, size)
    y[:,:,0] = rle
    plt.imsave(f"D:Dataset/dataset/hubmap-organ-segmentation/masked_images/{row['id']}.jpg", y)